# Обязательная часть
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [72]:
# imports
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from datetime import datetime

In [76]:
def scrap_habr(theme, page=1):
    params_req = {
        'url': 'https://habr.com/ru/search/',
        'search': {
            'target_type': 'posts',
            'order': 'relevance',
            'q': theme
    }}
    scrap_df = pd.DataFrame()
    for i in range(1,page+1):
        print('page', i)
        resp = req.get(params_req['url'] + f'page{i}/', params=params_req['search'])
        to_data = bs(resp.text)
        data = to_data.find_all('article', class_='tm-articles-list__item')
        for item in data:
            date = datetime.strptime(item.find('time')['title'],'%Y-%m-%d, %H:%M')
            title = item.find('h2', class_='tm-article-snippet__title').text
            link = 'https://habr.com' + item.find('a', class_='tm-article-snippet__title-link').get('href')
            likes = item.find('span', class_='tm-votes-meter__value_rating').text
            req_post = req.get(link)
            full_text = bs(req_post.text).find('div', class_='article-formatted-body').text
            row = {
                'date': date,
                'title': title,
                'link': link,
                'likes': likes,
                'full_text': full_text,
            }
            scrap_df = pd.concat([scrap_df, pd.DataFrame([row])])
            time.sleep(0.1)
        time.sleep(0.2)
    return scrap_df.reset_index(drop=True)

scrap_habr('python', 2)



page 1
page 2


,date,title,link,likes,full_text
0,2022-07-13 10:00:00,Сравнительный анализ эффективности работы Cyth...,https://habr.com/ru/post/676426/,+10,"Итак, что такое Cython? Cython – это язык, кот..."
1,2022-01-20 18:37:00,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,+10,"Курс нацелен дать максимальную пользу, поэтому..."
2,2021-12-13 09:00:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,+9,21 декабря Слёрм проведёт открытый урок «ChatO...
3,2020-04-21 18:35:00,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,+19,"\r\n20 апреля 2020 года, спустя почти десять л..."
4,2021-07-06 13:29:00,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,+3,"Первый открытый онлайн-митап сообщества, для к..."
5,2022-01-13 18:35:00,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,+10,19 января Слёрм проведёт открытый урок «Пишем ...
6,2020-12-04 21:03:00,Python как компилируемый статически типизирова...,https://habr.com/ru/news/t/531402/,+16,По данным широко известного в узких кругах Tio...
7,2022-03-08 12:13:00,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/t/654707/,0,Вышел выпуск расширения Python для Visual Stud...
8,2020-03-03 13:22:00,... начале этого года Python сместил Java и ...,https://habr.com/ru/company/itsumma/news/t/490...,+31,"Согласно отчету RedMonk за январь 2020 года, P..."
9,2021-11-16 16:09:00,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/company/epam_systems/news/...,+3,Компания EPAM запустила бесплатный курс для са...
